In [3]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
train=pd.read_csv("C:\\Users\\RANA ASIM\\Downloads\\archive (1)\\train.csv")
test=pd.read_csv("C:\\Users\\RANA ASIM\\Downloads\\archive (1)\\test.csv")
print("shape of train=",train.shape)
print("shape of test=",test.shape)

shape of train= (29451, 12)
shape of test= (68720, 11)


In [5]:
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2.0,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2.0,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2.0,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2.0,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2.0,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [6]:
X_train=train.drop(columns='TARGET(PRICE_IN_LACS)')
y_train=train['TARGET(PRICE_IN_LACS)']
X_test=test.copy()
print("shape of X_train=",X_train.shape)
print("shape of y_train=",y_train.shape)
print("shape of X_test=",test.shape)

shape of X_train= (29451, 11)
shape of y_train= (29451,)
shape of X_test= (68720, 11)


In [7]:
null_var=X_train.isnull().sum()
null_var

POSTED_BY              0
UNDER_CONSTRUCTION     0
RERA                   0
BHK_NO.               20
BHK_OR_RK             13
SQUARE_FT             38
READY_TO_MOVE          0
RESALE                 0
ADDRESS               26
LONGITUDE              8
LATITUDE              28
dtype: int64

In [8]:
num_var=X_train.select_dtypes(include=['int64','float64']).columns
num_var_miss=[var for var in num_var if null_var[var]>0]
num_var_miss

['BHK_NO.', 'SQUARE_FT', 'LONGITUDE', 'LATITUDE']

In [9]:
cat_var=X_train.select_dtypes(include='O').columns
cat_var_miss=[var for var in cat_var if null_var[var]>0]
cat_var_miss

['BHK_OR_RK', 'ADDRESS']

In [10]:
num_var_mean=['BHK_NO.', 'SQUARE_FT']
num_var_median=['LONGITUDE', 'LATITUDE']
cat_var_mode=['BHK_OR_RK']
cat_var_missing=['ADDRESS']
num_var_mean_imputer= Pipeline([('imputer', SimpleImputer(strategy='mean'))])
num_var_median_imputer=Pipeline([('imputer',SimpleImputer(strategy='median'))])
cat_var_imputer=Pipeline([('imputer',SimpleImputer(strategy='most_frequent'))])
cat_var_missing_imputer=Pipeline([('imputer',SimpleImputer( strategy='constant',fill_value='Missing'))])
                                                   

In [40]:
preprocessor= ColumnTransformer(transformers=[('mean_imputer',num_var_mean_imputer,num_var_mean),
                            ('median_impter',num_var_median_imputer,num_var_median),
                            ('mode_imputer',cat_var_imputer,cat_var_mode),
                            ('missing_imputer',cat_var_missing_imputer,cat_var_missing)])


In [41]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['BHK_NO.', 'SQUARE_FT']),
                                ('median_impter',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['LONGITUDE', 'LATITUDE']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['BHK_OR_RK']),
                                ('missing_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Missing',
                                                                strategy='constant'))]),
          

In [60]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['BHK_NO.', 'SQUARE_FT']),
                                ('median_impter',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['LONGITUDE', 'LATITUDE']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['BHK_OR_RK']),
                                ('missing_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Missing',
                                                      

In [61]:
preprocessor.named_transformers_['mean_imputer'].named_steps['imputer'].statistics_

array([2.39254528e+00, 1.98262089e+04])

In [44]:
train['BHK_NO.'].mean() #To check

2.392545275389895

In [45]:
preprocessor.named_transformers_['mode_imputer'].named_steps['imputer'].statistics_

array(['BHK'], dtype=object)

In [46]:
X_train_clean=preprocessor.transform(X_train)
X_test_clean=preprocessor.transform(X_test)

In [66]:
X_train_clean

array([[2.0, 1300.236407, 12.96991, 77.59796, 'BHK',
        'Ksfc Layout,Bangalore'],
       [2.0, 1275.0, 12.274538, 76.644605, 'BHK',
        'Vishweshwara Nagar,Mysore'],
       [2.0, 933.1597222, 12.778033, 77.632191, 'BHK',
        'Jigani,Bangalore'],
       ...,
       [2.0, 1022.641509, 26.928785, 75.828002, 'BHK',
        'Ajmer Road,Jaipur'],
       [2.0, 927.0790093, 12.90015, 80.22791, 'BHK',
        'Sholinganallur,Chennai'],
       [2.0, 896.7741935, 26.832353, 75.841749, 'BHK',
        'Jagatpura,Jaipur']], dtype=object)

In [63]:
#how much cloumns availaible in main dataframe
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['BHK_NO.', 'SQUARE_FT']),
 ('median_impter',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['LONGITUDE', 'LATITUDE']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['BHK_OR_RK']),
 ('missing_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='Missing', strategy='constant'))]),
  ['ADDRESS']),
 ('remainder', 'drop', [0, 1, 2, 6, 7])]

In [ ]:
preprocessor=ColumnTransformer()

In [64]:
X_train_clean_df=pd.DataFrame(X_train_clean, columns=num_var_mean+num_var_median+cat_var_mode+cat_var_missing)
X_train_clean_df.head()

,BHK_NO.,SQUARE_FT,LONGITUDE,LATITUDE,BHK_OR_RK,ADDRESS
0,2.0,1300.236407,12.96991,77.59796,BHK,"Ksfc Layout,Bangalore"
1,2.0,1275.0,12.274538,76.644605,BHK,"Vishweshwara Nagar,Mysore"
2,2.0,933.159722,12.778033,77.632191,BHK,"Jigani,Bangalore"
3,2.0,929.921143,28.6423,77.3445,BHK,"Sector-1 Vaishali,Ghaziabad"
4,2.0,999.009247,22.5922,88.484911,BHK,"New Town,Kolkata"


In [65]:
X_train_clean_df.isnull().sum()

BHK_NO.      0
SQUARE_FT    0
LONGITUDE    0
LATITUDE     0
BHK_OR_RK    0
ADDRESS      0
dtype: int64

In [50]:
X_train

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE
0,Owner,0,0,2.0,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960
1,Dealer,0,0,2.0,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605
2,Owner,0,0,2.0,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191
3,Owner,0,1,2.0,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500
4,Dealer,1,0,2.0,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911
...,...,...,...,...,...,...,...,...,...,...,...
29446,Owner,0,0,3.0,BHK,2500.000000,1,1,"Shamshabad Road,Agra",27.140626,78.043277
29447,Owner,0,0,2.0,BHK,769.230769,1,1,"E3-108, Lake View Recidency,,Vapi",39.945409,-86.150721
29448,Dealer,0,0,2.0,BHK,1022.641509,1,1,"Ajmer Road,Jaipur",26.928785,75.828002
29449,Owner,0,0,2.0,BHK,927.079009,1,1,"Sholinganallur,Chennai",12.900150,80.227910
